In [1]:
import tensorflow as tf
tf.config.experimental.set_memory_growth(
    tf.config.list_physical_devices('GPU')[0], True)
print(tf.__version__)
print("Usage of gpu: {}".format(tf.config.list_physical_devices()))
print("Built with CUDA: {}".format(tf.test.is_built_with_cuda()))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Init Plugin
2.5.0
Init Graph Optimizer
Init Kernel
Usage of gpu: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Built with CUDA: False
Num GPUs Available:  1


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow_addons as tfa
import tensorflow.keras
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, MaxPool2D
from sklearn.model_selection import KFold 
import pydot
import gc
import pickle


print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.5.0


In [3]:
base_dir = "/Users/tarun/IIITN Project/ADNI/"
root_dir = "./"
test_dir = base_dir + "test/"
train_dir = base_dir + "train/"
work_dir = root_dir + "dataset/"
save_dir = root_dir + "saved_models/"

In [4]:
IMG_SIZE = 256
IMAGE_SIZE = [256, 256]
DIM = (IMG_SIZE, IMG_SIZE)
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

In [5]:
work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)

train_data_gen = work_dr.flow_from_directory(directory=work_dir, target_size=DIM, batch_size=6500, shuffle=False)

Found 2614 images belonging to 5 classes.


In [21]:
train_data, train_labels = train_data_gen.next()
print(train_data.shape, train_labels.shape)
#Performing over-sampling of the data, since the classes are imbalanced
sm = SMOTE(random_state=42)
train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)
train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
print(train_data.shape, train_labels.shape)

(2614, 256, 256, 3) (2614, 5)
(5800, 256, 256, 3) (5800, 5)


In [22]:
#Splitting the data into train, test, and validation sets

train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

In [23]:
def construct_model(act='relu'):
    """Constructing a Sequential CNN architecture for performing the classification task. """
    
    model = Sequential([
        Input(shape=(*IMAGE_SIZE, 3)),
        Conv2D(16, 3, activation=act, padding='same'),
        MaxPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(32, 3, activation=act, padding='same'),
        MaxPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(64, 3, activation=act, padding='same'),
        MaxPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(128, 3, activation=act, padding='same'),
        MaxPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'),
        BatchNormalization(),
        Dropout(0.2),
        Conv2D(256, 3, activation=act, padding='same'),
        BatchNormalization(),
        Dropout(0.2),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.7),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(5, activation='softmax')        
    ], name = "cnn_model")

    return model

In [24]:
model = tensorflow.keras.models.load_model('/Users/tarun/IIITN Project/saved_models/model_13.h5')

In [25]:
test_scores = model.evaluate(val_data,val_labels)
print("Testing Accuracy: %.2f%%"%(test_scores[1] * 100))

2022-03-11 13:51:00.960885: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


29/29 [==============================] - 2s 64ms/step - loss: 0.5518 - acc: 0.8707 - auc: 0.9653 - f1_score: 0.8743
Testing Accuracy: 87.07%
